In [3]:
import sys
sys.path.append('../')

import sqlite3
from src.cmesrc.config import CMESRCV2_DB

In [2]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

try:
    cur.executescript("""
    ALTER TABLE HARPS_PIXEL_BBOX ADD COLUMN overlap_percent REAL;
    ALTER TABLE HARPS_PIXEL_BBOX ADD COLUMN overlap_harpnum INTEGER;
    """)
except sqlite3.OperationalError as e:
    print(f"Column already exists: {e}")

NameError: name 'sqlite3' is not defined

In [36]:
# So for some reason some columns have the x_min and x_max swapped?

cur.execute("""
            UPDATE HARPS_PIXEL_BBOX
            SET x_min = CASE WHEN x_min > x_max THEN x_max ELSE x_min END,
            x_max = CASE WHEN x_min > x_max THEN x_min ELSE x_max END
            WHERE x_min > x_max;
""")


In [37]:
cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_hpbb_harpnum_timestamp ON HARPS_PIXEL_BBOX (harpnum, timestamp);
CREATE INDEX IF NOT EXISTS idx_hpbb_harpnum ON HARPS_PIXEL_BBOX (harpnum);
CREATE INDEX IF NOT EXISTS idx_hpbb_timestamp ON HARPS_PIXEL_BBOX (timestamp);
CREATE INDEX IF NOT EXISTS idx_hpbb_bbox_coords ON HARPS_PIXEL_BBOX (x_min, x_max, y_min, y_max);
""")

ProgrammingError: Cannot operate on a closed database.

In [4]:
cur.executescript("""
-- Create an index on HARPS_PIXEL_BBOX for the join in the temp_overlap creation step
CREATE INDEX IF NOT EXISTS idx_hpb_harpnum_timestamp ON HARPS_PIXEL_BBOX(harpnum, timestamp);

-- Create temporary table with overlap info
CREATE TEMPORARY TABLE temp_overlap AS
SELECT
    a.harpnum AS harpnum1,
    b.harpnum AS harpnum2,
    a.timestamp AS timestamp,
    100.0 * CASE WHEN a.x_min < b.x_max AND a.x_max > b.x_min AND a.y_min < b.y_max AND a.y_max > b.y_min
        THEN (MIN(a.x_max, b.x_max) - MAX(a.x_min, b.x_min)) * (MIN(a.y_max, b.y_max) - MAX(a.y_min, b.y_min))
        ELSE 0
    END / NULLIF((a.x_max - a.x_min) * (a.y_max - a.y_min), 0) AS overlap_percent
FROM HARPS_PIXEL_BBOX a
JOIN HARPS_PIXEL_BBOX b ON a.timestamp = b.timestamp AND a.harpnum != b.harpnum;

-- Create an index on temp_overlap for the window function in the temp_largest_overlap creation step
CREATE INDEX IF NOT EXISTS idx_temp_overlap_harpnum1_timestamp_overlap ON temp_overlap(harpnum1, timestamp, overlap_percent DESC);

-- Create temporary table with largest overlap and corresponding harpnum
CREATE TEMPORARY TABLE temp_largest_overlap AS
WITH ranked AS (
    SELECT harpnum1, timestamp, overlap_percent, harpnum2,
           ROW_NUMBER() OVER(PARTITION BY harpnum1, timestamp ORDER BY overlap_percent DESC) AS row_num
    FROM temp_overlap
    WHERE overlap_percent > 0
)
SELECT harpnum1, timestamp, overlap_percent AS max_overlap_percent, harpnum2 AS max_harpnum2
FROM ranked
WHERE row_num = 1;

-- Create an index on temp_largest_overlap for the join in the UPDATE HARPS_PIXEL_BBOX step
CREATE INDEX IF NOT EXISTS idx_temp_largest_overlap_harpnum1_timestamp ON temp_largest_overlap(harpnum1, timestamp);

-- Update the HARPS_PIXEL_BBOX table using a temporary table and a common table expression
WITH updated AS (
  SELECT h.rowid, t.max_overlap_percent, t.max_harpnum2
  FROM HARPS_PIXEL_BBOX h
  JOIN temp_largest_overlap t ON h.harpnum = t.harpnum1 AND h.timestamp = t.timestamp
)
UPDATE HARPS_PIXEL_BBOX
SET overlap_percent = (SELECT max_overlap_percent FROM updated u WHERE u.rowid = HARPS_PIXEL_BBOX.rowid),
    overlap_harpnum = (SELECT max_harpnum2 FROM updated u WHERE u.rowid = HARPS_PIXEL_BBOX.rowid)
WHERE rowid IN (SELECT rowid FROM updated);

-- Drop the temporary tables
DROP TABLE temp_overlap;
DROP TABLE temp_largest_overlap;

""")

In [8]:
conn.commit()
conn.close()

In [9]:
conn.close()

In [10]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

try:
    cur.executescript("""
    ALTER TABLE HARPS_BBOX ADD COLUMN overlap_percent REAL;
    ALTER TABLE HARPS_BBOX ADD COLUMN overlap_harpnum INTEGER;
    """)
except sqlite3.OperationalError as e:
    print(f"Column already exists: {e}")


In [11]:
cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_hbb_harpnum_timestamp ON HARPS_BBOX (harpnum, timestamp);
CREATE INDEX IF NOT EXISTS idx_hbb_harpnum ON HARPS_BBOX (harpnum);
CREATE INDEX IF NOT EXISTS idx_hbb_timestamp ON HARPS_BBOX (timestamp);
CREATE INDEX IF NOT EXISTS idx_hbb_bbox_coords ON HARPS_BBOX (LONDTMIN, LONDTMAX, LATDTMIN, LATDTMAX);
""")

In [16]:
cur.executescript("""
-- Create an index on HARPS_BBOX for the join in the temp_overlap creation step
CREATE INDEX IF NOT EXISTS idx_hbb_harpnum_timestamp ON HARPS_BBOX(harpnum, timestamp);

-- Create temporary table with overlap info
CREATE TEMPORARY TABLE temp_overlap AS
SELECT
    a.harpnum AS harpnum1,
    b.harpnum AS harpnum2,
    a.timestamp AS timestamp,
    100.0 * CASE WHEN a.LONDTMIN < b.LONDTMAX AND a.LONDTMAX > b.LONDTMIN AND a.LATDTMIN < b.LATDTMAX AND a.LATDTMAX > b.LATDTMIN
        THEN (MIN(a.LONDTMAX, b.LONDTMAX) - MAX(a.LONDTMIN, b.LONDTMIN)) * ABS((SIN(0.0174532925 * MIN(a.LATDTMAX, b.LATDTMAX)) - SIN(0.0174532925 * MAX(a.LATDTMIN, b.LATDTMIN))))
        ELSE 0
    END / NULLIF((a.LONDTMAX - a.LONDTMIN) * ABS(SIN( 0.0174532925 * a.LATDTMAX) - SIN(0.0174532925 * a.LATDTMIN)), 0) AS overlap_percent
FROM HARPS_BBOX a
JOIN HARPS_BBOX b ON a.timestamp = b.timestamp AND a.harpnum != b.harpnum;

-- Create an index on temp_overlap for the window function in the temp_largest_overlap creation step
CREATE INDEX IF NOT EXISTS idx_temp_overlap_harpnum1_timestamp_overlap ON temp_overlap(harpnum1, timestamp, overlap_percent DESC);

-- Create temporary table with largest overlap and corresponding harpnum
CREATE TEMPORARY TABLE temp_largest_overlap AS
WITH ranked AS (
    SELECT harpnum1, timestamp, overlap_percent, harpnum2,
           ROW_NUMBER() OVER(PARTITION BY harpnum1, timestamp ORDER BY overlap_percent DESC) AS row_num
    FROM temp_overlap
    WHERE overlap_percent > 0
)
SELECT harpnum1, timestamp, overlap_percent AS max_overlap_percent, harpnum2 AS max_harpnum2
FROM ranked
WHERE row_num = 1;

-- Create an index on temp_largest_overlap for the join in the UPDATE HARPS_BBOX step
CREATE INDEX IF NOT EXISTS idx_temp_largest_overlap_harpnum1_timestamp ON temp_largest_overlap(harpnum1, timestamp);

-- Update the HARPS_BBOX table using a temporary table and a common table expression
WITH updated AS (
  SELECT h.rowid, t.max_overlap_percent, t.max_harpnum2
  FROM HARPS_BBOX h
  JOIN temp_largest_overlap t ON h.harpnum = t.harpnum1 AND h.timestamp = t.timestamp
)
UPDATE HARPS_BBOX
SET overlap_percent = (SELECT max_overlap_percent FROM updated u WHERE u.rowid = HARPS_BBOX.rowid),
    overlap_harpnum = (SELECT max_harpnum2 FROM updated u WHERE u.rowid = HARPS_BBOX.rowid)
WHERE rowid IN (SELECT rowid FROM updated);

-- Drop the temporary tables
DROP TABLE temp_overlap;
DROP TABLE temp_largest_overlap;

""")

In [17]:
conn.commit()
conn.close()

In [22]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

try:
    cur.execute("ALTER TABLE HARPS_BBOX ADD COLUMN area REAL;")
except sqlite3.OperationalError as e:
    print(f"Column already exists: {e}")

Column already exists: duplicate column name: area


In [23]:

cur.executescript("""
-- Create an index on HARPS_BBOX for the join in the temp_overlap creation step
CREATE INDEX IF NOT EXISTS idx_hbb_harpnum_timestamp ON HARPS_BBOX(harpnum, timestamp);

CREATE INDEX IF NOT EXISTS idx_hbb_harpnum_timestamp ON HARPS_BBOX(harpnum, timestamp);

UPDATE HARPS_BBOX
SET area = NULLIF(100 * 0.0174532925 * (LONDTMAX - LONDTMIN) * ABS(SIN(0.0174532925 * LATDTMAX) - SIN(0.0174532925 * LATDTMIN)) / (2 * 3.141592653), 0);
""")

In [24]:
conn.commit()
conn.close()

In [33]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

# Drop area column
cur.execute("ALTER TABLE HARPS_PIXEL_BBOX DROP COLUMN area;")

try:
    cur.execute("ALTER TABLE HARPS_PIXEL_BBOX ADD COLUMN area REAL;")
except sqlite3.OperationalError as e:
    print(f"Column already exists: {e}")

In [34]:

cur.executescript("""
-- Create an index on HARPS_PIXEL_BBOX for the join in the temp_overlap creation step
CREATE INDEX IF NOT EXISTS idx_hpbb_harpnum_timestamp ON HARPS_PIXEL_BBOX(harpnum, timestamp);

CREATE INDEX IF NOT EXISTS idx_hpbb_harpnum_timestamp ON HARPS_PIXEL_BBOX(harpnum, timestamp);

UPDATE HARPS_PIXEL_BBOX
SET area = 100 * (x_max - x_min) * (y_max - y_min) / (512 * 512);
""")

In [35]:
conn.commit()
conn.close()